In [ ]:
import os
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
drive.mount('/content/drive')
assets_dir = '/content/drive/MyDrive/dp_nature'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_global_antero = pd.read_csv(os.path.join(assets_dir, 'mpfc_retrograde_global_fractions_all_merge.csv')).drop(columns='Unnamed: 818')
df_global_antero

,Case ID,Injection Site,GU_d_i,GU_d_c,GU_s_i,GU_s_c,PTLp_d_i,PTLp_d_c,PTLp_s_i,PTLp_s_c,...,CUL_mo_i,CUL_mo_c,IP_i,IP_c,DN_i,DN_c,FN_i,FN_c,TOTAL_i,TOTAL_c
0,Merged,DPs,0.001035,0.000000,0.000583,0.000130,0.001054,0.000087,0.000214,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.920250,0.071311
1,Merged,DPd,0.007141,0.001253,0.002380,0.000125,0.002130,0.000251,0.000125,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.929216,0.062140
2,Merged,ILA,0.006659,0.000181,0.003310,0.000000,0.001200,0.000091,0.005463,0.000890,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.869944,0.128664
3,Merged,PL,0.007326,0.000321,0.000478,0.000191,0.007267,0.001095,0.010561,0.000533,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.780736,0.218654
4,Merged,ACAv,0.002391,0.000245,0.000245,0.000245,0.011850,0.000122,0.013587,0.002268,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.734797,0.265203
5,Merged,ACAd,0.014593,0.000444,0.000000,0.000000,0.017558,0.001174,0.026358,0.000951,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.764950,0.234270


In [ ]:
df_antero_long = df_global_antero.melt(id_vars=['Case ID', 'Injection Site'], value_vars=df_global_antero.columns[2:], var_name='ROI', value_name='fraction')
df_antero_long = df_antero_long.assign(hemisphere=df_antero_long.ROI.map(lambda roi: 'ipsilateral' if roi.endswith('_i') else 'contralateral'),
                                       ROI=df_antero_long.ROI.str.slice(0, -2))
df_antero_long = df_antero_long.loc[df_antero_long.ROI != 'TOTAL', :]
df_antero_long

,Case ID,Injection Site,ROI,fraction,hemisphere
0,Merged,DPs,GU_d,0.001035,ipsilateral
1,Merged,DPd,GU_d,0.007141,ipsilateral
2,Merged,ILA,GU_d,0.006659,ipsilateral
3,Merged,PL,GU_d,0.007326,ipsilateral
4,Merged,ACAv,GU_d,0.002391,ipsilateral
...,...,...,...,...,...
4879,Merged,DPd,FN,0.000000,contralateral
4880,Merged,ILA,FN,0.000000,contralateral
4881,Merged,PL,FN,0.000000,contralateral
4882,Merged,ACAv,FN,0.000000,contralateral


In [ ]:
df_hierarchy = pd.read_csv(os.path.join(assets_dir, 'tables', 'roi_hierachy.csv'), header=None, names=['fine_roi', 'color_code', 'hierarchy', 'coarse_roi', 'ROI'])
df_hierarchy

,fine_roi,color_code,hierarchy,coarse_roi,ROI
0,border_11,0,BORDER,border_11,border_11
1,GU_6b,40048,CTX,GU,GU_d
2,GU_6a,40049,CTX,GU,GU_d
3,GU_5,40050,CTX,GU,GU_d
4,GU_4,40051,CTX,GU,GU_d
...,...,...,...,...,...
825,CUL_mo,16776337,HB,CUL_mo,CUL_mo
826,IP,16776593,HB,IP,IP
827,DN,16776849,HB,DN,DN
828,FN,16777105,HB,FN,FN


In [ ]:
df_antero_long = df_antero_long.set_index('ROI').join(df_hierarchy.set_index('ROI')[['hierarchy']], how='left').reset_index()
df_antero_long

,ROI,Case ID,Injection Site,fraction,hemisphere,hierarchy
0,A13,Merged,DPs,0.0,ipsilateral,HY
1,A13,Merged,DPd,0.0,ipsilateral,HY
2,A13,Merged,ILA,0.0,ipsilateral,HY
3,A13,Merged,PL,0.0,ipsilateral,HY
4,A13,Merged,ACAv,0.0,ipsilateral,HY
...,...,...,...,...,...,...
7111,sAMY,Merged,DPd,0.0,contralateral,AMY
7112,sAMY,Merged,ILA,0.0,contralateral,AMY
7113,sAMY,Merged,PL,0.0,contralateral,AMY
7114,sAMY,Merged,ACAv,0.0,contralateral,AMY


In [ ]:
# EP_v is spelled Ep_v in hierarchy table
# Some df_antero_long.ROI map to hierarchy table coarse roi.
df_antero_long_na_hierarchy = df_antero_long.loc[df_antero_long.hierarchy.isna()]
for _, row in df_antero_long_na_hierarchy.iterrows():
  hierarchy_row = df_hierarchy.loc[df_hierarchy.coarse_roi.str.lower() == row.ROI.lower()]
  if hierarchy_row.empty: continue
  df_antero_long.loc[df_antero_long.ROI.str.lower() == row.ROI.lower(), 'hierarchy'] = hierarchy_row.hierarchy.values[0]

In [ ]:
# Check all rows are assigned hierarchy.
df_antero_long.loc[df_antero_long.hierarchy.isna()]

,ROI,Case ID,Injection Site,fraction,hemisphere,hierarchy


In [ ]:
TOP_PERCENT = 0.1

df_antero_max = df_antero_long.groupby('ROI')['fraction'].max().sort_values(ascending=False)
df_antero_max = df_antero_max[0: int(len(df_antero_max) * TOP_PERCENT)]
df_antero_max

ROI
PIR_s       0.274128
ACAv_s      0.101214
ENTl_s      0.101074
SUB_v       0.093084
CA1         0.091276
PIR_d       0.075252
RE          0.071916
MOs_d       0.071294
ACAv_d      0.063826
BLA         0.062478
CLA         0.060189
EP_d        0.056936
MOs_s       0.053276
MOp_s       0.040631
ACAd_d      0.040210
RSPv_s      0.036562
AM_d        0.036309
COApm_d     0.036081
ACAd_s      0.035590
ENTl_d      0.031696
PL_s        0.030886
MD_I        0.027338
ECT_s       0.026447
PTLp_s      0.026358
RSPv_d      0.026262
ORBvl_s     0.025958
PVT         0.025933
RSPagl_s    0.023746
COApm_s     0.023678
PL_d        0.023396
ILA_s       0.021924
PERI_s      0.021783
VISam_s     0.020976
AON_m       0.020421
ORBm_s      0.019548
VM          0.018524
AIv_s       0.018417
VISam_d     0.018353
MOp_d       0.018277
SUB_d       0.018259
Name: fraction, dtype: float64

In [ ]:
hierarchy_order = {'CTX': 0, 'STR': 1, 'AMY': 2, 'TH': 3, 'HY': 4, 'HPF': 5, 'MB': 6, 'HB': 7, 'WHITE_MATTER': 8, 'VENTRICLE': 9, 'BORDER': 10}
df_antero_long_strong = df_antero_long.loc[df_antero_long.ROI.isin(df_antero_max.index)].sort_values(by='ROI').sort_values(by='hierarchy', key=lambda hierarchy: hierarchy.map(hierarchy_order), kind='stable')
df_antero_long_strong

,ROI,Case ID,Injection Site,fraction,hemisphere,hierarchy
24,ACAd_d,Merged,DPs,0.003616,ipsilateral,CTX
50,ACAd_d,Merged,DPs,0.000152,contralateral,CTX
51,ACAd_d,Merged,DPs,0.000152,contralateral,CTX
52,ACAd_d,Merged,DPd,0.000626,contralateral,CTX
54,ACAd_d,Merged,DPd,0.000626,contralateral,CTX
...,...,...,...,...,...,...
6052,SUB_v,Merged,ACAv,0.003076,ipsilateral,HPF
6050,SUB_v,Merged,ILA,0.062423,ipsilateral,HPF
6049,SUB_v,Merged,DPd,0.093084,ipsilateral,HPF
6048,SUB_v,Merged,DPs,0.000779,ipsilateral,HPF


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

injection_site = 'ACAv'
tracer = 'retrograde'
df_injection = df_antero_long_strong.loc[df_antero_long_strong['Injection Site'] == injection_site]
df_ipsi = df_injection.loc[df_injection.hemisphere == 'ipsilateral']
df_contra = df_injection.loc[df_injection.hemisphere == 'contralateral']

plt.figure(figsize=(100, 150))
plt.title(f"{injection_site} {tracer}", fontdict={'fontsize': 80})
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xlabel('fraction', fontsize=70)
plt.ylabel('ROI', fontsize=70)
plt.rc('legend',fontsize=60)
_ = sns.barplot(x="fraction", y="ROI", data=df_ipsi, label="ipsilateral", errorbar=None)
_ = sns.barplot(x="fraction", y="ROI", data=df_contra, label="contralateral", errorbar=None)